In [4]:
import pandas as pd
import numpy as np


In [5]:
base_prediction = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/data_science/predictions.parquet")
base_prediction = base_prediction[["y_true", "y_pred_s1_50_gen_parallel"]]
base_prediction

,y_true,y_pred_s1_50_gen_parallel
2022-12-01 22:44:00+00:00,-70.484,-70.529247
2022-12-01 22:59:00+00:00,-125.662,11.984345
2022-12-01 23:14:00+00:00,71.946,-159.113161
2022-12-01 23:29:00+00:00,-101.859,24.945586
2022-12-01 23:44:00+00:00,-45.071,-14.599998
...,...,...
2023-06-30 20:29:00+00:00,-61.305,-25.595774
2023-06-30 20:44:00+00:00,-13.549,-6.777618
2023-06-30 20:59:00+00:00,-29.614,9.842615
2023-06-30 21:14:00+00:00,-40.824,-43.734448


In [6]:
spline = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/hour_spline_encoded.parquet")
sin_cos = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/hour_sin_cos_encoded.parquet")
onehot = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/hour_onehot_encoded.parquet")
ordinal = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/hour_ordinal_encoded.parquet")